In [1]:
import re
import math
import random
import time
import corenlp
import nltk
from nltk.tokenize import sent_tokenize
import codecs

In [2]:
dimensionality=2000
denseness=10//dimensionality

In [3]:
def sparseadd(onevec,othvec,weight=1,normalised=False):
    if normalised:
        onevec = normalise(onevec)
        othvec = normalise(othvec)        
    result={}
    try:
        for l in onevec:
            result[l] = onevec[l]
        for k in othvec:
            if k in result:
                result[k] = result[k]+othvec[k]*float(weight)
            else:
                result[k] = othvec[k]*float(weight)
    except:
        print("sparseadd(): error")
        raise
    return result
def sparsemultiply(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            if l in othvec:
                result[l] = onevec[l]*othvec[l]*float(weight)
    except:
        print("sparsemultiply(): error ")
    return result
def sparsexor(onevec,othvec):
    result={}
    try:
        for l in range(len(onevec)):
            if ((l in onevec) and not (l in othvec)):
                result[l] = 1
            if (not (l in onevec) and (l in othvec)):
                result[l] = 1        
    except:
        print("sparsexor(): error ")
    return result

def newrandomvector(n,denseness):
    vec = {}
    k = int(n * denseness)
    if k % 2 != 0:
        k += 1
    if (k > 0):# no need to be careful about this, right? and k % 2 == 0):
        nonzeros = random.sample(list(range(n)),k)
        negatives = random.sample(nonzeros,k//2)
        for i in nonzeros:
            vec[str(i)] = 1;
        for i in negatives:
            vec[str(i)] = -1;
    return vec

def newoperator(n):
    k = 0.1
    return newrandomvector(n,k)

def sparsecosine(xvec,yvec,rounding=True,decimals=4):
    x2 = 0
    y2 = 0
    xy = 0
    try:
        for i in xvec:
            x2 += xvec[i]*xvec[i]
    except KeyError:
        print("sparsecosine(): error at position ",i)
    try:
        for j in yvec:
            y2 += yvec[j]*yvec[j]
            if j in xvec:
                xy += xvec[j]*yvec[j]
    except:
        print("sparsecosine(): errors at position ",j)
    if (x2*y2 == 0):
        cos = 0
    else:
        cos = xy/(math.sqrt(x2)*math.sqrt(y2))
    if (rounding):
        cos=round(cos,decimals)
    return cos

def sparselength(vec,rounding=True):
    x2 = 0
    length=0
    try:
        for i in vec:
            x2 += vec[i]*vec[i]
    except:
        print("sparselength(): error at position ",i)
    if (x2 > 0):
        length = math.sqrt(x2)
    if (rounding):
        length=round(length,4)
    return length

def comb(vec,k=0.1):
    newvector={}
    n=int(k*dimensionality/2)
    sorted_items=sorted(vec.items(), key=lambda x:x[1])
    bot=sorted_items[:n]
    top=sorted_items[-n:]
    for l in bot:
        newvector[l[0]]=l[1]
    for l in top:
        newvector[l[0]]=l[1]
    return newvector

def sparsesum(vec):
    s=0
    for i in vec:
        s += float(vec[i])
    return s

def normalise(vec):
    newvector={}
    vlen=sparselength(vec,False)
    if (vlen > 0):
        for i in vec:
            newvector[i]=vec[i]/math.sqrt(vlen*vlen)
    else:
        newvector=vec
    return newvector

def modify(vec,factor):
    newvector={}
    for i in vec:
        if (random.random() > factor):
            newvector[i]=vec[i]
        else:
            newvector[i]=float(vec[i])*(0.5-random.random())*2.0
    return newvector

def createpermutation(k):
    permutation=random.sample(range(k), k)
    return permutation
    
def permute(vector,permutation):
    newvector={}
    try:
        for i in range(len(permutation)):
            if str(i) in vector:
                newvector[str(permutation[i])]=vector[str(i)]
    except:
        newvector=vector
        print("permute(): no permutation done, something wrong")
    return newvector

def vectorsaturation(vector):
    d = 0
    for c in vector:
        d += 1
    return d

def frequencyweight(word):
    try:
        w = math.exp(-300*math.pi* globalfrequency[word] / bign)
    except KeyError:
        w = 0.5
    return w

In [ ]:
wordspacefile="/home/jussi/data/wordspaces/gavagai-news-22-morethan5.wordspace"
fdgtextfile="/home/jussi/data/GH95-fdg/950102.sgml.fdg.xml"
unittestfile="/home/jussi/data/cooksents.txt"


In [4]:
class clause:
    cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
    def __init__(self,string):
        self.surfacestring=string
        self.cleanedutterance=re.sub(clause.cleanuppattern,"",string)
        self.tokens=self.cleanedutterance.lower().split()
        self.agent=None
        self.event=None
        self.patient=None
        self.instrument=None
        self.location=None
        self.manner=None
    def __str__(self):
        return self.surfacestring
    
class referent:
    def __init__(self, string):
        self.surfacestring=string
        self.definite=True
        self.number=1
        
class event:
    def __init__(self, string):
        self.surfacestring=string    
        self.negated=False    
        self.adverbial=None    
        self.tense=None #past present or future
        self.aspect=None #ongoing perfect or pointwise
        self.mood=None #indicative, irreal, potential, optative, not, imperative

In [21]:
#from nltk.parse.stanford import StanfordDependencyParser
#path_to_stanford_corenlp = '/usr/share/stanford-corenlp-full/'
#parser_jar='stanford-corenlp.jar'
#path_to_models = path_to_stanford_corenlp+'models/'
#model_jar='stanford-english-corenlp-models.jar'
#dependency_parser = StanfordDependencyParser(path_to_jar=path_to_stanford_corenlp+parser_jar, path_to_models_jar=path_to_models+model_jar)

parser_client = corenlp.CoreNLPClient(annotators="tokenize ssplit pos natlog lemma depparse".split())
    
    
def semanticdepparse(string,debug=False):
    depgraph=parser_client.annotate(string)
    utterances=[]
    for ss in depgraph.sentence:
        utterances.append(semanticdepparseprocess(string,ss,debug))
    return utterances
        
def semanticdepparseprocess(string,ss,debug=False):
    utterance=clause(string)
    utterance.agent=referent("epsilon")
    utterance.patient=referent("epsilon")
    utterance.event=event("epsilon")
    scratch={}
    negated=False
    adverbial=None
    if debug:
 #       print(ss)
        i=1
        print("root:",ss.basicDependencies.root)
        for w in ss.token:
            print(i," ",w.lemma," ",w.pos)
            i += 1
        for e in ss.basicDependencies.edge:
            print(e.source,ss.token[e.source-1].lemma,"-",e.dep,"->",e.target,ss.token[e.target-1].lemma) 
    i=1
    sentenceitems={}
    sentencepos={}
    for w in ss.token:
        if w.lemma not in indexspace:
            indexspace[w.lemma] = newrandomvector(dimensionality,denseness)
        sentenceitems[i] = w.lemma
        sentencepos[i] = w.pos
        scratch[i] = False
        i += 1
    root = ss.basicDependencies.root[0] #only one root for now fix this!
    utterance.event=event(sentenceitems[root])  
    utterance.event.tense="PRESENT"
    if sentencepos[root] == "VBD":
        utterance.event.tense="PAST" 
    if sentencepos[root] == "VBN":
        utterance.event.tense="PAST" 
    for edge in ss.basicDependencies.edge:
        if debug:
            print(edge.source,sentenceitems[edge.source],"-",edge.dep,"->",edge.target,sentenceitems[edge.target])
        if edge.dep == 'neg':
            negated=True
        elif edge.dep == 'nsubj':
            utterance.agent=referent(sentenceitems[edge.target])
            if edge.target in scratch: 
                if scratch[edge.target]=="def":
                    utterance.agent.definite=True
                elif scratch[edge.target]=="indef":
                    utterance.agent.definite=False
        elif edge.dep == 'dobj':
                utterance.patient=referent(sentenceitems[edge.target])
                if edge.target in scratch: 
                    if scratch[edge.target]=="def":
                        utterance.patient.definite=True
                    elif scratch[edge.target]=="indef":
                        utterance.patient.definite=False
        elif edge.dep == 'advmod':
                adverbial=sentenceitems[edge.target]            
        elif edge.dep == 'det':
                if (sentenceitems[edge.target]=="a" or sentenceitems[edge.target]=="an"):
                    scratch[edge.source]="indef"
                elif (sentenceitems[edge.target]=="the"):
                    scratch[edge.target]="def"
        elif edge.dep == 'nmod:poss':
                scratch[edge.target]="def"
        elif edge.dep == 'aux':
                if (sentenceitems[edge.target]=="have"):
                    scratch['aux']="have"
                if (sentenceitems[edge.target]=="do"):
                    scratch['aux']="do"
                    if (sentencepos[edge.target] == "VBD"):
                        utterance.event.tense = "PAST"
                if (sentenceitems[edge.target]=="will"):
                    scratch['aux']="will"
                if (sentenceitems[edge.target]=="shall"):
                    scratch['aux']="shall"
    try:
        if debug: 
            print(sentenceitems[root]," ",sentencepos[root])
        if sentencepos[root] == "VB" and (scratch['aux'] == "will" or scratch['aux'] == "shall"):
            utterance.event.tense="FUTURE"
    except:
        print("semanticdepparse(): tense situation in "+string)
    try:
        utterance.event.adverbial=adverbial
    except:
        print("semanticdepparse(): adverbial blowout in "+string)
    try:
        utterance.event.negated=negated
    except:
        print("semanticdepparse(): negation mismatch in "+string)
    return utterance


In [ ]:
vectorspace={}
globalfrequency={}
indexspace={}
associationspace={}

def chkwordspace(words,debug=False):
    i = 0
    for w in words:
        if w in vectorspace:
                i += 1
        else:
            if debug:
                print("chkwordspace(): ",w," is new and is now hallucinated.")
            denseness2=0.5
            vectorspace[w] = newrandomvector(dimensionality,denseness2)
            indexspace[w] = newrandomvector(dimensionality,denseness)
            associationspace[w] = newrandomvector(dimensionality,denseness2)  
            globalfrequency[w] = 1

chkwordspace(["epsilon"],True)

def wordlikeness(w1,w2):
    return sparsecosine(vectorspace[w1],vectorspace[w2])

def dowordspacefile(wordspacefile):
    linepattern = re.compile(r'^\(\"([\-\w]+)\"\s+#S2000([\d\,\-\.\-\+;e]+)*:\s+#S2000([\d\-\+;\.e\,]+)*:\s+#S2000([\.\d\-\+;e\,]+)*:\s+(\d+)\)')
    vecpattern = re.compile(r';?(\d+)([\+\-][\,\.\d]+(e-\d\d)?)')
    alicefile=open(wordspacefile,'r')
    global vectorspace
    vectorspace={}
    global globalfrequency
    globalfrequency={}
    global indexspace
    indexspace={}
    global associationspace
    associationspace={}
    aliceline = alicefile.readline()
    a=1
    b=0
    w=0
    global bign = 0
    while aliceline:
        a = a+1
        m=linepattern.match(aliceline)
        if m:
            token=m.groups()[0]
            ind=m.groups()[1]
            doc=m.groups()[2]
            ctx=m.groups()[3]
            freq=int(m.groups()[4])
#            if (filtering and token )
            try:
                sparsevec={}
                cells = re.findall(vecpattern,ctx)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                vectorspace[token] = sparsevec
                sparsevec={}
                cells = re.findall(vecpattern,ind)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                indexspace[token] = sparsevec
                globalfrequency[token] = freq
                sparsevec={}
                w=w+1
                bign += freq
            except:
                print("fail: ",token," ",a)
        else:
            b=b+1
        aliceline = alicefile.readline()
    print(a," lines seen; ",w," items added ",b," items disregarded")
    alicefile.close()
    

In [ ]:
dowordspacefile(wordspacefile)

In [ ]:
utterancespace = {}
dependencyanalysisstore={}
def initunittest(debug=False):
    with codecs.open(fulltextfile,"r", encoding='utf-8') as infile:
        rawtext = infile.read().lower()
        rawtext = re.sub('\n', ' ', rawtext)
        rawtext = re.sub('\"', ' ', rawtext)
        rawtext = re.sub('\s+', ' ', rawtext)
        sents = sent_tokenize(rawtext)
        i = 0
        for s in sents:
            print(s)
            wds=nltk.word_tokenize(s)
            chkwordspace(wds,debug)
            try:
                analyses = semanticdepparse(s.lower(),debug)
                dependencyanalysisstore[s]=analyses
                kk = 0
                for analysis in analyses:
                    u = utterancevector(analysis,debug)
                    if (kk == 0):
                        utterancespace[s] = u
                    else:
                        uttterancespace[s+str(kk)] = u
                    kk += 1
                i += 1
            except:
                print("****",s)
                              

In [ ]:
fulltextfile="/home/jussi/data/pan12-clean/clean.12AtrainA1.txt"
initunittest(False)

In [ ]:
def rununittest():
    for v in utterancespace:
        print("-------------")
        print(v)
        neighbours = {}
        for w in utterancespace:
            if w == v:
                continue
            neighbours[w] = sparsecosine(utterancespace[v],utterancespace[w])
        ns = sorted(neighbours.items(), key=lambda neighbour:neighbour[1], reverse=True)[0:10]
        for ww in ns:
                print("\t",ww)

In [ ]:
target = {}
target = sparseadd(target,permute(negationabstract,negationpermutation))
wd = "victor"
target = sparseadd(target,comb(vectorspace[wd]),math.log(bign / globalfrequency[wd]))
for w in utterancespace:
    cc = sparsecosine(utterancespace[w],target) 
#    if (cc > 0.1):
    if wd in w:
        print(cc," ",w)


In [ ]:
sparsecosine(vectorspace["book"],vectorspace["books"])

In [ ]:
test_grep()

In [18]:
def utterancevector(utterance,debug=False,lexical=True,morphology=True,semanticroles=True,constructional=True):
    baremorphology=morphology
    combinedmorphology=lexical
    uttvec={}
    if debug:
        debugvec={}
    if lexical:     # add in referent, using its context vector meaning more or less the concept it refers to
        if (utterance.patient):
            uttvec=sparseadd(uttvec, comb(vectorspace[utterance.patient.surfacestring]),frequencyweight(utterance.patient.surfacestring)) #, math.log( bign / globalfrequency[utterance.patient.surfacestring]))
        if (utterance.event):
            uttvec=sparseadd(uttvec,comb(vectorspace[utterance.event.surfacestring]),frequencyweight(utterance.event.surfacestring)) #, math.log( bign / globalfrequency[utterance.event.surfacestring]))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,comb(vectorspace[utterance.agent.surfacestring]),frequencyweight(utterance.agent.surfacestring)) #, math.log( bign / globalfrequency[utterance.agent.surfacestring]))
    if debug:
        print("utterancevec() lexicon: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if morphology:     # add in morphological information about the roles of the referent
        morphvec={}
        if (utterance.agent):
            if (utterance.agent.definite):
                morphvec=sparseadd(morphvec,permute(indexspace[utterance.agent.surfacestring],definitepermutation))
            else:
                morphvec=sparseadd(morphvec,permute(indexspace[utterance.agent.surfacestring],indefinitepermutation))
        if (utterance.patient):
            if (utterance.patient.definite):
                    morphvec=sparseadd(morphvec,permute(indexspace[utterance.patient.surfacestring],definitepermutation))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                    morphvec=sparseadd(morphvec,permute(indexspace[utterance.patient.surfacestring],indefinitepermutation))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                    morphvec=sparseadd(morphvec,permute(indexspace[utterance.event.surfacestring],futuretensepermutation))
            if (utterance.event.tense=="PAST"):
                    morphvec=sparseadd(morphvec,permute(indexspace[utterance.event.surfacestring],pasttensepermutation))
            else:
                    morphvec=sparseadd(morphvec,permute(indexspace[utterance.event.surfacestring],presenttensepermutation))
        uttvec=sparseadd(normalise(uttvec),normalise(morphvec))
        if debug:
            print("utterancevec() morphology: ",sparsecosine(uttvec,debugvec))
            debugvec=uttvec
    if semanticroles:
        semroles = {}
        if (utterance.patient):
                semroles=sparseadd(semroles,permute(indexspace[utterance.patient.surfacestring],patientpermutation))
        if (utterance.event):
                semroles=sparseadd(semroles,permute(indexspace[utterance.event.surfacestring],eventpermutation))
        if (utterance.event.negated):
                semroles=sparseadd(semroles,permute(indexspace[utterance.event.surfacestring],negationpermutation))
        if (utterance.event.adverbial):
                semroles=sparseadd(semroles,permute(indexspace[utterance.event.adverbial],adverbialpermutation))
        if (utterance.agent):
                semroles=sparseadd(semroles,permute(indexspace[utterance.agent.surfacestring],agentpermutation))
        uttvec=sparseadd(normalise(uttvec),normalise(semroles))
        if debug:
            print("utterancevec() semanticrole: ",sparsecosine(uttvec,debugvec))
            debugvec=uttvec
    if constructional:    # add in morphological information about the roles, no account of referent
        constrvec={}
        if (utterance.agent):
            if (utterance.agent.definite):
                constrvec=sparseadd(constrvec,permute(agentabstract,definitepermutation))
            else:
                constrvec=sparseadd(constrvec,permute(agentabstract,indefinitepermutation))
        if (utterance.patient):
            if (utterance.patient.definite):
                constrvec=sparseadd(constrvec,permute(patientabstract,definitepermutation))
            else:
                constrvec=sparseadd(constrvec,permute(patientabstract,indefinitepermutation))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                constrvec=sparseadd(constrvec,permute(futureabstract,tensepermutation))
            elif (utterance.event.tense=="PAST"):
                constrvec=sparseadd(constrvec,permute(pastabstract,tensepermutation))
            else:
                constrvec=sparseadd(constrvec,permute(presentabstract,tensepermutation)) 
        if (utterance.event.negated):
            uttvec=sparseadd(normalise(uttvec),normalise(permute(negationabstract,negationpermutation)))
        uttvec=sparseadd(normalise(uttvec),normalise(constrvec))
        if debug:
            print("utterancevec() constructions: ",sparsecosine(uttvec,debugvec))
            debugvec=uttvec
    return uttvec


In [13]:
def initconstantsandpermutations(dimensionality=2000,densenessforabstracts=0.20):

    global definitepermutation
    global indefinitepermutation
    global futurepermutation
    global pasttensepermutation
    global presenttensepermutation
    global agentpermutation
    global patientpermutation
    global eventpermutation
    global instrumentpermutation
    global locationpermutation
    global mannerpermutation
    global negationpermutation
    global adverbialpermutation

    global previouspermutation

    global tensepermutation

    global agentabstract
    global patientabstract
    global eventabstract
    global instrumentabstract
    global locationabstract
    global mannerabstract
    global negationabstract

    global presentabstract
    global pastabstract
    global progressiveabstract
    global futureabstract

    definitepermutation=createpermutation(dimensionality)
    indefinitepermutation=createpermutation(dimensionality)
    futurepermutation=createpermutation(dimensionality)
    pasttensepermutation=createpermutation(dimensionality)
    presenttensepermutation=createpermutation(dimensionality)
    agentpermutation=createpermutation(dimensionality)
    patientpermutation=createpermutation(dimensionality)
    eventpermutation=createpermutation(dimensionality)
    instrumentpermutation=createpermutation(dimensionality)
    locationpermutation=createpermutation(dimensionality)
    mannerpermutation=createpermutation(dimensionality)
    negationpermutation=createpermutation(dimensionality)
    adverbialpermutation=createpermutation(dimensionality)
    #sequence
    previouspermutation=createpermutation(dimensionality)

    tensepermutation=createpermutation(dimensionality)

    agentabstract=newrandomvector(dimensionality,densenessforabstracts)
    patientabstract=newrandomvector(dimensionality,densenessforabstracts)
    eventabstract=newrandomvector(dimensionality,densenessforabstracts)
    instrumentabstract=newrandomvector(dimensionality,densenessforabstracts)
    locationabstract=newrandomvector(dimensionality,densenessforabstracts)
    mannerabstract=newrandomvector(dimensionality,densenessforabstracts)
    negationabstract=newrandomvector(dimensionality,densenessforabstracts)

    presentabstract=newrandomvector(dimensionality,densenessforabstracts)
    pastabstract=newrandomvector(dimensionality,densenessforabstracts)
    progressiveabstract=newrandomvector(dimensionality,densenessforabstracts)
    futureabstract=newrandomvector(dimensionality,densenessforabstracts)


In [ ]:
def command_loop(debug=False,moredebug=False):
    instring = ''
    prev=newrandomvector(dimensionality,denseness)
    prevlex=newrandomvector(dimensionality,denseness)
    prevsem=newrandomvector(dimensionality,denseness)
    prevmor=newrandomvector(dimensionality,denseness)
    prevmgn=newrandomvector(dimensionality,denseness)
    instring = input('> ')
    while instring != 'quit':
        try:
            s = instring.rstrip()
            wds=nltk.word_tokenize(s.lower())
            chkwordspace(wds,debug)
            try:
                cs = semanticdepparse(s.lower(),debug)
                for c in cs:
                    u = utterancevector(c,moredebug, True, False, False,False)
                    print(" lexical     ",sparsecosine(prev,u),end="\t(")
                    print(sparsecosine(prevlex,u),")")
                    prevlex=u
                    u = utterancevector(c,moredebug, False, False, True, False)
                    print(" semrole     ",sparsecosine(prev,u),end="\t(")
                    print(sparsecosine(prevsem,u),")")
                    prevsem=u
                    u = utterancevector(c,moredebug,  False, True, False, False)
                    print(" morph     ",sparsecosine(prev,u),end="\t(")
                    print(sparsecosine(prevmor,u),")")
                    prevmor=u
                    u = utterancevector(c,moredebug, False, False, False, True)
                    print(" construction  ",sparsecosine(prev,u),end="\t(")
                    print(sparsecosine(prevmgn,u),")")
                    prevmgn=u
                    u = utterancevector(c,moredebug)
                    print(" in toto     ",sparsecosine(prev,u))
                    prev = u
            except:
                print("****")
            instring = input('> ')
        except:
            instring = 'quit'
    print("hey!")
    
    

In [ ]:
command_loop(False,False)



In [10]:
# lexicon: what concepts are mentioned (context vectors)
f1 = False

# morphology: what morphological form is represented (index vectors permuted with morph permutations)
f2 = True

# semantic roles: e.g. is the agent definite? (index vectors, deep case permutations)
f3 = True

# constructions: (abstract random feature vectors)
f4 = True

#debug output (no good at present)
d = False
a = "The cat eats the cream."
b = "The fish ate the worm."
c = "she winked."
target=permute(pastabstract,tensepermutation)
ap = semanticdepparse(a.lower())[0]
bp = semanticdepparse(b.lower())[0]
cp = semanticdepparse(c.lower())[0]
av = utterancevector(ap,d,f1,f2,f3,f4)
bv = utterancevector(bp,d,f1,f2,f3,f4)
cv = utterancevector(cp,d,f1,f2,f3,f4)

print("a vs b\t",sparsecosine(av,bv))
print("a vs c\t",sparsecosine(av,cv))
print("b vs c\t",sparsecosine(bv,cv))

print("a vs tgt\t",sparsecosine(av,target))
print("b vs tgt\t",sparsecosine(bv,target))
print("c vs tgt\t",sparsecosine(cv,target))


NameError: name 'pastabstract' is not defined

In [ ]:
cp.event.tense


In [ ]:
initunittest()

# test negation

## how soon will a vector be saturated?
results: the densensess of the vector has no bearing on the quality of the retrieval. i have tested here with denseness stepped from 0.1 to 1 of a randomised abstract vector. all come out with same retrieval cosine. but the cosine varies for different tasks. need to check why.

In [20]:
initconstantsandpermutations(dimensionality,0.2)
indexspace={}


In [23]:
#for denseness in [0.05]: #[0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]:


#for k in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
#    negationabstract=newrandomvector(dimensionality,k)
#    presentabstract=newrandomvector(dimensionality,k)

def uv(words):
    utterance=semanticdepparse(words.lower())[0]
    return utterancevector(utterance,False, False, True, True, True)


print("definite:\t",sparsecosine(uv("The cat does not eat the fish."),permute(agentabstract,definitepermutation)))
print("definite:\t",sparsecosine(uv("The cat does not eat the fish."),permute(patientabstract,definitepermutation)))
print("negation:\t",sparsecosine(uv("The cat does not eat the fish."),permute(negationabstract,negationpermutation)))
print("non-neg vs negation:\t",sparsecosine(uv("The cat eats the fish."),permute(negationabstract,negationpermutation)))
print("present:\t",sparsecosine(uv("The cat eats the fish."),permute(presentabstract,tensepermutation)))
print("past vs present:\t",sparsecosine(uv("The cat has not eaten the fish."),permute(presentabstract,tensepermutation)))
print("neg vs non-neg:\t",sparsecosine(uv("The cat eats the fish."),uv("The cat did not eat the fish.")))
print("similar lex:\t",sparsecosine(uv("The cat eats the fish."),uv("The dog eats the fish.")))
print("similar lex:\t",sparsecosine(uv("The cat eats the fish."),uv("The fish ate the worm.")))

#initconstantsandpermutations(dimensionality,0.2)


definite:	 0.3973
definite:	 0.4102
negation:	 0.6985
non-neg vs negation:	 -0.0241
present:	 0.5793
past vs present:	 0.0323
neg vs non-neg:	 0.4826
similar lex:	 1.0
similar lex:	 0.6808


# Can we find indefinite agents?

In [ ]:
for i in utterancespace:
    c = sparsecosine(utterancespace[i],vectorspace["hand"])  #sparseadd(,normalise(permute(negationabstract,negationpermutation)))) #sparseadd(permute(negationabstract,negationpermutation),permute(agentabstract,definitepermutation)))) #,permute(negationabstract,negationpermutation)))
    if (c > 0.1):
        d = sparsecosine(utterancespace[i],permute(negationabstract,negationpermutation))
        print (c, d,i)
        if (d > 0.1):
            print (c, d,i)

# weighting

now an exponential, math.exp(200*math.pi*bign/globalfrequency(wd)) which defaults to 0.5 if unknown word, i'd prefer arc tan tho

# Two new spaces, using known context vectors, one weighted one not

In [ ]:
newspace = {}
newweightedspace = {}
target = "victor"
for uu in utterancespace:
    wds=nltk.word_tokenize(uu.lower())
    if target in wds:
        newtestvector = {}
        newweightedtestvector = {}
        for wd in wds:
            try:
                n1 = newtestvector
                newtestvector = sparseadd(newtestvector,normalise(comb(vectorspace[wd])),1,False)
                print(wd," ",wds," ",sparsecosine(n1,newtestvector))
                newweightedtestvector = sparseadd(newweightedtestvector,normalise(comb(vectorspace[wd])),frequencyweight(wd),False)
            except KeyError:
                pass
        newspace[uu] = newtestvector
        newweightedspace[uu] = newweightedtestvector

# use the new space, for each word in an utterance, how close is it?

In [ ]:
probewd = "victor"

for uu in newspace:
    wds=nltk.word_tokenize(uu.lower())
    print("============================")
    print(uu)
    if probewd in wds:
        for wd in wds:
            try:
                cc = sparsecosine(newspace[uu],comb(vectorspace[wd]))
                ccw = sparsecosine(newweightedspace[uu],comb(vectorspace[wd]))
                print(cc,"\t",ccw,"\t",wd,"\t",round(frequencyweight(wd),2),"\t",uu)
            except KeyError:
                pass

In [ ]:
def test_grep(debug=False,moredebug=False):
    instring = ''
    instring = input('> ')
    prev=newrandomvector(dimensionality,denseness)
    prevstring = "null"
    ryggrad=permute(negationabstract,negationpermutation)
    while instring != 'quit':
        try:
            s = instring.rstrip()
            wds=nltk.word_tokenize(s.lower())
            chkwordspace(wds,debug)
            try:
                cs = semanticdepparse(s.lower(),debug)
                for c in cs:
                    u = utterancevector(c,moredebug)
                    print(" cosine between ",wds," and ",prevstring," :     ",sparsecosine(prev,u))
                    print(" cosine between ",wds," and negation :     ",sparsecosine(ryggrad,u))
                    prev = u
                    prevstring = wds
            except:
                print("****")
            instring = input('> ')
        except:
            instring = 'quit'
    print("hey!")
    
    

In [ ]:
def sparserecoverabilityexperiment():
    step = 20
    start = 10
    stop = 150
    acceptable = 0.2
    for cellsize in [2,3,4,9,16,25,36,64,100]:
        print("=============================================")
        print("number of features aggregated: ",cellsize)
        print("analytic optimum: ",(2*cellsize*dimensionality)**(-1/3)*dimensionality)

        #cellsize = 9 # how many features in an utterance (includes words)
        d = start
        featspace = {} # a hash of cellsize random vectors of denseness d
        featvec = {} # an addition of cellsize random vectors of denseness d into one vector
        bigs = 0
        ii = 0
        while (d <= stop):
            i = 0
            featspace[d] = [None]*cellsize
            featvec[d] = {}
            while (i < cellsize): 
                featspace[d][i] = newrandomvector(dimensionality,d / dimensionality)
                prev = featvec[d]
                featvec[d] = sparseadd(featvec[d],featspace[d][i])
#            print(d,i,sparsecosine(prev,featvec[d]),sparsecosine(featvec[d],featspace[d][i]),sep="\t")
                i += 1
            i = 0
            lils = 0
            #esses = 0
            while (i < cellsize):
                c = sparsecosine(featvec[d],featspace[d][i])
#            print(d,i,sparselength(featspace[d][i]),sep="\t")
            #if c < acceptable:
            #    esses += 1
                lils += c
                i += 1
            avs = lils / cellsize
#        print(d,i,sparselength(featvec[d]),sep="\t")
            print(d,end="\t", flush=True)
#        print("\tfeature: ","{0:.3f}".format(avs)," (",esses,")", flush=True)
            print("\tfeature: ","{0:.3f}".format(avs), flush=True)
            ii += i
            bigs += lils
            d += step
        avs = bigs / ii
        print("average retrievability: ","{0:.4f}".format(avs))    

In [ ]:
sparserecoverabilityexperiment()
